In [7]:
#极坐标弧长曲线求解部分
from scipy import integrate
import sympy as sp
import numpy as np


# 定义符号变量theta
theta = sp.symbols('theta')

#化简积分表达式
c = 170 / (2 * sp.pi)

# 弧长的不定积分,单位为cm
int_expr = 0.5 * c * (sp.sqrt(1 + theta**2) * theta + sp.asinh(theta))

int_expr


42.5*(theta*sqrt(theta**2 + 1) + asinh(theta))/pi

In [8]:
from numba import jit

def arc(t0,arc_length) -> float:
    '''
    输入当前的角度t0和弧长arc_length
    返回前进到的角度t1
    积分是从t1到t0，因为t1的角度更小,前进过程中角度是在减小的
    '''
    a =  0.5 * c * (sp.sqrt(1 + t0**2) * t0 + sp.asinh(t0))
    t1 = sp.symbols('t1') # 定义符号变量t1,设定为未知数
    b =  0.5 * c * (sp.sqrt(1 + t1**2) * t1 + sp.asinh(t1))
    
    equation = a -b - arc_length
    #sp.pprint(equation)
    result = sp.nsolve(equation,t1, t0)
    
    return float(result)

'''
当前角度为32π，沿着弧线前进2cm
print(arc(32*np.pi,1))
'''

class Queue:
    '''
    实现一个队列，先入先出(FIFO)
    '''
    def __init__(self):
        self.queue = []
    
    def is_empty(self): 
        return len(self.queue) == 0
    
    def push(self, element):
        self.queue.append(element)
    
    def pop(self):
        if self.is_empty():
            return None
        else:
            return self.queue.pop(0)
    
    def get_size(self):
        return len(self.queue)

        
def search_vertex(head_theta) -> list:
    c = 170 / (2 * np.pi)  # 常数 c，用来计算螺线坐标
    theta_step = 0.00001     # 角度步长
    vertex = Queue()      # 队列，用于存储待处理的节点
    vertex_list = []      # 存储所有找到的顶点角度
    
    # 初始角度作为第一个顶点
    vertex.push(head_theta)
    vertex_list.append(head_theta)
    while not vertex.is_empty():
        print(len(vertex_list))
        if len(vertex_list) > 223: # 限制最多只能找到 224 个顶点
            print("找完了",len(vertex_list))
            break
        
        # 从队列中取出当前的角度（作为线段的固定端点）
        current_theta = vertex.pop()
        #print(f"Processing theta: {current_theta}")
        
        # 当前点的坐标
        x0 = c * current_theta * np.cos(current_theta)
        y0 = c * current_theta * np.sin(current_theta)
        
        # 从当前 theta 开始，移动下一个点
        theta = current_theta
        while theta <= 32 * np.pi:
            # 尝试计算下一个点的角度
            next_theta = theta + theta_step
            x1 = c * next_theta * np.cos(next_theta)
            y1 = c * next_theta * np.sin(next_theta)
            
            # 计算两点之间的距离
            distance = np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2)
            #print(distance)
            if distance > 890:
                print(f"Warning!!!!! Distance too large: {distance}")
            
            # 处理距离为 286 的情况，且这是寻找“头部”的条件
            if distance >= 286 and len(vertex_list) == 1:
                #print("ok")
                #print(f"Found head: {next_theta} with distance: {distance}")
                vertex_list.append(next_theta)
                vertex.push(next_theta)  # 将新找到的顶点作为新的固定端点继续搜索
                break  # 找到符合条件的点后跳出当前循环，开始处理下一个顶点
            
            # 处理距离为 165 的情况
            if distance >= 165 and len(vertex_list) > 1 :
                #print(f"Found vertex at theta: {next_theta} with distance: {distance}")
                vertex_list.append(next_theta)
                vertex.push(next_theta)  # 将新找到的顶点作为新的固定端点继续搜索
                break  # 找到符合条件的点后跳出当前循环，开始处理下一个顶点
            
            # 更新 theta，继续向下搜索
            theta += theta_step
    
    return vertex_list
'''
# 测试函数
found_vertices = search_vertex(0.0)
print("Found vertices:", found_vertices)
'''
def distance(a,b) -> float:
    '''
    输入两个角度a和b
    用近似替代方法计算两点之间的距离
    '''
    c = 170 / (2 * np.pi)
    x0 = c * a * np.cos(a)
    y0 = c * a * np.sin(a)
    x1 = c * b * np.cos(b)
    y1 = c * b * np.sin(b)
    #print("x0,y0,x1,y1",x0,y0,x1,y1)
    return np.sqrt((x1 - x0) ** 2 + (y1 - y0) ** 2)
#distance(32.67299999997398, 33.25299999997675)


def distance_q(a, b) -> float:
    c = np.float64(170) / (2 * np.float64(np.pi))
    
    p = np.float64(1) / np.float64(2) * c * (np.sqrt(np.float64(1) + np.float64(a)**np.float64(2)) * np.float64(a) + np.arcsinh(np.float64(a)))
    q = np.float64(1) / np.float64(2) * c * (np.sqrt(np.float64(1) + np.float64(b)**np.float64(2)) * np.float64(b) + np.arcsinh(np.float64(b)))
    
    return np.abs(p - q)


def get_speed(t0,time):
    '''
    输入初始角度和运动时间time
    确定时间微元dt为1e-7
    '''
    dt = 1
    
    current_arc = time*200 #当前头部的弧长
    updated_arc = (time+dt)*200 #dt时间内头部加上走过的弧长
    
    head_theta = arc(t0,current_arc) #头部当前的角度
    updated_head_theta = arc(t0,updated_arc) #dt后头部的角度
    
    current_theta_list = search_vertex(head_theta)
    #print(current_theta_list)
    after_theta_list = search_vertex(updated_head_theta)  
    #print(after_theta_list)
    
    speed_list = []
    for i in range(len(current_theta_list)):
        a = current_theta_list[i]
        b = after_theta_list[i]
        ds = distance(a,b)
        speed = ds/dt
        speed_list.append(speed)
        
    return current_theta_list,speed_list


def get_speed_q(t0,time):
    '''
    输入初始角度和运动时间time
    确定时间微元dt为1e-7
    '''
    
    dt = 1e-2
    
    current_arc = time*200 #当前头部的弧长
    updated_arc = (time+dt)*200 #dt时间内头部加上走过的弧长
    
    head_theta = arc(t0,current_arc) #头部当前的角度
    updated_head_theta = arc(t0,updated_arc) #dt后头部的角度
    
    current_theta_list = search_vertex(head_theta)
    print(current_theta_list)
    after_theta_list = search_vertex(updated_head_theta)  
    print(after_theta_list)
    
    speed_list = []
    for i in range(len(current_theta_list)):
        a = current_theta_list[i]
        b = after_theta_list[i]
        ds = distance_q(a,b)
        speed = ds/dt
        speed_list.append(speed)
        
    return current_theta_list,speed_list


#print(get_speed(32*np.pi,400)[1])
print(get_speed_q(32*np.pi,400)[1])
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
找完了 224
[64.75591523984565, 64.91911523989745, 65.01301523992726, 65.10677523995702, 65.20040523998674, 65.29389524001641, 65.38725524004604, 65.48048524007564, 65.57357524010519, 65.66653524013469, 65.7593652401

In [9]:

print(distance(88.8117400996444, 89.0241140991082))
print(distance(88.81174008672816, 89.02411408619197))
print(float(1e-7*100))

510.00040089752815
510.0004008234547
9.999999999999999e-06


In [10]:
#用于特定的时间点
def q1_test():
    def mian_q1(time):
        theta_list , speed_list = get_speed(32*np.pi,time)
        print(len(theta_list),theta_list)
        print(len(speed_list),speed_list)

        target_list = [0,1,51,101,151,201,223]
        x_list = []
        y_list = []
        target_speed = []
        for i in target_list:
            try:
                temp = theta_list[i]
                x = 55 / (2 * np.pi) * temp * np.cos(temp)
                y = 55 / (2 * np.pi) * temp * np.sin(temp)
                print(f"theta: {temp}, x: {x}, y: {y}")
                x_list.append(x)
                y_list.append(y)
                target_speed.append(speed_list[i])
            except IndexError:
                print(f"IndexError: {i}")
                
        print("x:",x_list)
        print("y:",y_list)
        print("speed:",target_speed)

        import pandas as pd
        df = pd.DataFrame({"target_list":target_list[0:len(x_list)],"x":[o / 100 for o in x_list],"y":[p/100 for p in y_list],"speed":[q/100 for q in target_speed]})
        #df.to_csv(f"q1_{time}.csv",index=False)

    time_list = [0,60,120,180,240,300]
    for i in time_list:
        mian_q1(i)
        print("time:",i)
        print("---------------------------------------------------")


In [11]:
import pandas as pd
def q1():
    def main_q1(time):
        theta_list , speed_list = get_speed(32*np.pi,time)
        print(len(theta_list),theta_list)
        print(len(speed_list),speed_list)
        x_list = []
        y_list = []
        target_speed = []
        for i in range(len(theta_list)):
                temp = theta_list[i]
                x = 55 / (2 * np.pi) * temp * np.cos(temp)
                y = 55 / (2 * np.pi) * temp * np.sin(temp)
                print(f"theta: {temp}, x: {x}, y: {y}")
                x_list.append(x)
                y_list.append(y)
                target_speed.append(speed_list[i])
                
        print("x:",x_list)
        print("y:",y_list)
        print("speed:",target_speed)

        import pandas as pd
        df = pd.DataFrame({"x":[o/100 for o in x_list],"y":[p/100 for p in y_list]})
        
        x = df["x"].tolist()
        y = df["y"].tolist()
        
        alternating_list = [val for pair in zip(x, y) for val in pair]
        df_alternating = pd.DataFrame({time:alternating_list})
        df_alternating.to_csv(f"{time}.csv",index=True)
        print(df_alternating)

    for i in range(300+1):
        main_q1(i)
        print("time:",i)
        print("--------------------------------------------------")
        
def q1_speed():
    
    def main_q1(time):
        theta_list , speed_list = get_speed(np.pi*32,time)
        df = pd.DataFrame({f"{time}":[o/100 for o in speed_list]})
        return df
    
    for i in range(0,300+1):
        df = main_q1(i)
        df.to_csv(f"{i}.csv",index=True)
        print("time:",i)

#q1_speed()
        

In [12]:

#画出图
from matplotlib import pyplot as plt
import numpy as np
import math
plt.rcParams['text.usetex'] = True #设置使用latex渲染

def draw(time):

    fig,ax = plt.subplots(1,1,figsize=(5,5),dpi = 400,subplot_kw={'projection':'polar'})

    theta = np.linspace(0,32*np.pi,10000)
    r =(55/(2*np.pi))*theta
    ax.grid(False)
    ax.set_yticklabels([])

    # 设置外圈的度数显示为latex格式
    angles = np.arange(0, 360, 45)
    angle_labels = [f"${angle}^\\circ$" for angle in angles]
    ax.set_thetagrids(angles, labels=angle_labels)


    position_data = get_speed(32*np.pi,time)[0]
    print(position_data)

    for i in position_data:
        ax.scatter(i,(55/(2*np.pi))*i,color = 'blue',s = 5,marker='*')

    # 将所描的相邻点用线连接起来
    for i in range(len(position_data) - 1):
        ax.plot([position_data[i], position_data[i + 1]], 
                [(55 / (2 * np.pi)) * position_data[i], (55 / (2 * np.pi)) * position_data[i + 1]], 
                color='red', linewidth=1)

    ax.plot(theta,r,linewidth=0.5)
    plt.savefig(f"q1_{time}.svg",format='svg')
    plt.show()
'''   
for i in np.arange(0,300+1,0.5):
    draw(i)
    print("time:",i)
    print("--------------------------------------------------")
'''
#draw(300)

'   \nfor i in np.arange(0,300+1,0.5):\n    draw(i)\n    print("time:",i)\n    print("--------------------------------------------------")\n'